# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
惑星形成における2変数ダスト合体成長数値計算とその高速化	Talk	talk-2024-planetwakate	惑星科学若手研究会2024	2024-03-04	オンライン	https://sites.google.com/view/psw2024
ツリー法を応用した多成分ダストの合体成長計算アルゴリズム	Talk	talk-2025-wakate	2025年度 第55回天文・天体物理若手夏の学校	2025-07-30	長野県千曲市 戸倉上山田温泉 ホテル圓山荘	https://astro-wakate.sakura.ne.jp/ss2025	星間現象/星・惑星形成分科会のオーラルアワードで1位を受賞
ツリー法を応用した多成分ダストの合体成長計算アルゴリズム	Poster	poster-2025-wakusei	日本惑星科学会2025年秋季講演会	2025-09-03	東京都 東京大学 駒場Iキャンパス	https://www.wakusei.jp/meetings/fallmeeting/2025/index.html	ポスター発表
ツリー法を応用した多成分ダストの合体成長計算アルゴリズム	Poster	poster-2025-tenmonb	日本天文学会2025年秋季年会	2025-09-09	山口県下関市 海峡メッセ下関	https://www.asj.or.jp/nenkai/archive/2025b/	ポスター発表
An O(d N^d log N) Tree Algorithm for the Multi-Component Coagulation Equation	Talk	talk-2025-eanam10	East Asia Numerical Astrophysics Meetings 10 (EANAM10)	2025-09-16	Sinbuk-ro, Jocheon-eup, Jeju-si, Jeju-do, South Korea	https://coma.kasi.re.kr/eanam10/php/list.php	国際学会にて口頭発表

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,惑星形成における2変数ダスト合体成長数値計算とその高速化,Talk,talk-2024-planetwakate,惑星科学若手研究会2024,2024-03-04,オンライン,https://sites.google.com/view/psw2024,NaN
1,ツリー法を応用した多成分ダストの合体成長計算アルゴリズム,Talk,talk-2025-wakate,2025年度 第55回天文・天体物理若手夏の学校,2025-07-30,長野県千曲市 戸倉上山田温泉 ホテル圓山荘,https://astro-wakate.sakura.ne.jp/ss2025,星間現象/星・惑星形成分科会のオーラルアワードで1位を受賞
2,ツリー法を応用した多成分ダストの合体成長計算アルゴリズム,Poster,poster-2025-wakusei,日本惑星科学会2025年秋季講演会,2025-09-03,東京都 東京大学 駒場Iキャンパス,https://www.wakusei.jp/meetings/fallmeeting/20...,ポスター発表
3,ツリー法を応用した多成分ダストの合体成長計算アルゴリズム,Poster,poster-2025-tenmonb,日本天文学会2025年秋季年会,2025-09-09,山口県下関市 海峡メッセ下関,https://www.asj.or.jp/nenkai/archive/2025b/,ポスター発表
4,An O(d N^d log N) Tree Algorithm for the Multi...,Talk,talk-2025-eanam10,East Asia Numerical Astrophysics Meetings 10 (...,2025-09-16,"Sinbuk-ro, Jocheon-eup, Jeju-si, Jeju-do, Sout...",https://coma.kasi.re.kr/eanam10/php/list.php,国際学会にて口頭発表


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2024-03-04-talk-2024-planetwakate.md 2025-09-09-poster-2025-tenmonb.md
2025-07-30-talk-2025-wakate.md       2025-09-16-talk-2025-eanam10.md
2025-09-03-poster-2025-wakusei.md


In [7]:
!cat ../_talks/2025-07-30-talk-2025-wakate.md

---
title: "ツリー法を応用した多成分ダストの合体成長計算アルゴリズム"
collection: talks
type: "Talk"
permalink: /talks/2025-07-30-talk-2025-wakate
venue: "2025年度 第55回天文・天体物理若手夏の学校"
date: 2025-07-30
location: "長野県千曲市 戸倉上山田温泉 ホテル圓山荘"
---

[More information here](https://astro-wakate.sakura.ne.jp/ss2025)

星間現象/星・惑星形成分科会のオーラルアワードで1位を受賞
